In [1]:
from typing import List, Tuple
import torch
import numpy as np
import pandas as pd

/Users/lino/opt/anaconda3/envs/datascience/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def mode_of_freq(data: pd.DataFrame,
                 key='date',
                 freq='D',
                 mode='sum'
                 ) -> pd.DataFrame:
    # 日付をobjectからdate_time型に変更
    data.date = pd.to_datetime(data.date, format=('%d.%m.%Y'))
    # 時系列(key)について日毎(D)の売上数の合計値(sum)で出力
    mode_of_key = getattr(data.groupby(pd.Grouper(key=key, freq=freq)), mode)

    return mode_of_key()


def making_dataset(
                ds: pd.Series,
                span=7,
                ) -> List[Tuple[np.ndarray, np.ndarray]]:
    data = ds.copy()
    endpoint = len(data) - span - 2
    src = np.stack([data[i: i+span] for i in range(0, endpoint)])
    trg = np.stack([data[i+span-1: i+span+1] for i in range(0, endpoint)])
    out = np.stack([data[i+span: i+span+2] for i in range(0, endpoint)])
    pack = (src, trg, out)
    pack = [torch.from_numpy(i.astype(np.float32)).clone() for i in pack]
    return pack

In [13]:
data = pd.read_csv('../data/sales_train.csv')
data = mode_of_freq(data).item_cnt_day
pack = making_dataset(data)

In [14]:
pack[0]

tensor([[1951., 8198., 7422.,  ..., 6346., 5858., 4984.],
        [8198., 7422., 6617.,  ..., 5858., 4984., 4772.],
        [7422., 6617., 6346.,  ..., 4984., 4772., 3162.],
        ...,
        [1711., 1647., 1632.,  ..., 3035., 2495., 1387.],
        [1647., 1632., 2901.,  ..., 2495., 1387., 1551.],
        [1632., 2901., 3035.,  ..., 1387., 1551., 3593.]])

In [15]:
pack[1]

tensor([[4984., 4772.],
        [4772., 3162.],
        [3162., 3142.],
        ...,
        [1387., 1551.],
        [1551., 3593.],
        [3593., 1589.]])

In [16]:
pack[2]

tensor([[4772., 3162.],
        [3162., 3142.],
        [3142., 3829.],
        ...,
        [1551., 3593.],
        [3593., 1589.],
        [1589., 2274.]])